# Tuning NSGA-II hyperparameters on ZDT1

This notebook demonstrates how to run the built-in tuner to search NSGA-II hyperparameters on the ZDT1 problem.

In [1]:
%load_ext autoreload
%autoreload 2
import json
from pathlib import Path
import numpy as np

from vamos.engine.tuning import tune

from vamos.foundation.problem.registry import make_problem_selection
from vamos.foundation.core.runner import ExperimentConfig

# Set a deterministic seed for reproducibility
SEED = 42
OUT_DIR = Path("results/tuning_zdt1")
OUT_DIR.mkdir(parents=True, exist_ok=True)


In [2]:
# Problem selection
selection = make_problem_selection("zdt1", n_var=30)
# Larger inner budget for serious tuning
base_config = ExperimentConfig(population_size=200, max_evaluations=40000, seed=SEED)

from vamos.engine.tuning import build_nsgaii_config_space

# Full hyperparameter space for NSGA-II (all continuous operators)
config_space = build_nsgaii_config_space()

print(config_space)


In [3]:
# Run tuning (outer NSGA-II optimizing HV of inner NSGA-II configs)
# Higher meta budget for real search
tuning_results = tune(
    problem_selection=selection,
    base_config=base_config,
    config_space=config_space,
    n_generations=120,  # meta generations
    population_size=60,  # meta population
    seed=SEED,
    output_dir=OUT_DIR,
    verbose=True,
)

print(f"Evaluated {len(tuning_results['trials'])} configurations")


KeyboardInterrupt: 

In [ ]:
# Inspect and summarize tuning results
import json

def _to_serializable(obj):
    import numpy as np
    if hasattr(obj, 'to_dict'):
        return _to_serializable(obj.to_dict())
    if isinstance(obj, dict):
        return {k: _to_serializable(v) for k, v in obj.items()}
    if isinstance(obj, (list, tuple)):
        return [_to_serializable(v) for v in obj]
    if isinstance(obj, np.generic):
        return obj.item()
    return obj

trials_raw = tuning_results['trials']
print(f'Evaluated {len(trials_raw)} configurations (including duplicates)')

# Deduplicate by config to make the summary clearer
seen = set()
trials = []
for t in trials_raw:
    cfg_key = json.dumps(_to_serializable(t['config']), sort_keys=True)
    if cfg_key in seen:
        continue
    seen.add(cfg_key)
    trials.append(t)

print(f'Unique configurations: {len(trials)}')

# Sort by HV (meta objective stores negative HV in slot 0)
sorted_trials = sorted(trials, key=lambda t: t['objective'][0])
top_k = sorted_trials[:5]

def _summarize(cfg):
    pop = cfg.get('pop_size')
    cross = cfg.get('crossover', ['-', {}])
    mut = cfg.get('mutation', ['-', {}])
    return f"pop={pop}, cross={cross[0]}, mut={mut[0]}"

print('Top configs by HV:')
for rank, trial in enumerate(top_k, 1):
    obj = trial['objective']
    hv = -obj[0]
    time = obj[1]
    robust = obj[2]
    cfg = trial['config']
    summary = _summarize(cfg)
    print(f"#{rank}: HV={hv:.4f}, time={time:.3f}, robust={robust:.3f} | {summary}")

best = sorted_trials[0]
best_cfg = _to_serializable(best['config'])
best_hv = -best['objective'][0]
print('\nBest configuration details:')
print(f"HV={best_hv:.4f}")
print(json.dumps(best_cfg, indent=2))

# Save all trials (optional)
# trials_path = OUT_DIR / 'tuning_trials.json'
# with open(trials_path, 'w', encoding='utf-8') as fh:
#     json.dump(_to_serializable(tuning_results), fh, indent=2)
# print(f'Trials saved to {trials_path}')


Evaluated 60 configurations (including duplicates)
Unique configurations: 30
Top configs by HV:
#1: HV=14.1013, time=1.185, robust=0.000 | pop=80, cross=spx, mut=uniform_reset
#2: HV=13.4265, time=1.166, robust=0.000 | pop=80, cross=spx, mut=uniform_reset
#3: HV=13.1617, time=1.165, robust=0.000 | pop=80, cross=spx, mut=uniform_reset
#4: HV=10.9392, time=1.159, robust=0.000 | pop=80, cross=spx, mut=uniform_reset
#5: HV=10.3129, time=1.147, robust=0.000 | pop=80, cross=spx, mut=uniform_reset

Best configuration details:
HV=14.1013
{
  "pop_size": 80,
  "crossover": [
    "spx",
    {
      "prob": 0.8258544383597095,
      "epsilon": 0.261752260749445
    }
  ],
  "mutation": [
    "uniform_reset",
    {
      "prob": "1/n"
    }
  ],
  "selection": [
    "tournament",
    {
      "pressure": 2
    }
  ],
  "survival": "nsga2",
  "engine": "numpy",
  "offspring_size": null,
  "repair": null,
  "archive": null
}
